# Installation

Let's start by building a basic chatbot.For conversation history, we require a CloudSQL instance running in a Google
Cloud project that the user or service account that executes your LangChain
code is authenticated with. Look at the Google Cloud documentation for more
information about [setting up and connecting to CloudSQL instances](https://cloud.google.com/sql/docs/mysql/connect-instance-cloud-shell).

These libraries contain the core components of LangChain, as well as the ability to
handle memory in a local MySQL database

In [ ]:
!pip install -qU ipykernel
!pip install -U pip

In [ ]:
# Base components for using LangChain on Google Cloud
! pip install -qU langchain-google-vertexai
# LangGraph enables creation of graph-based chat agents
! pip install -qU langgraph httpx
# Grandalf is a tool to visualize graphs (like LangGraph)
! pip install -qU grandalf # for visualizing the graph
# Enables LangGraph to use SQLite for memory handling
! pip install -qU langgraph-checkpoint-sqlite # for chatbot memory

In [3]:
# If you are using Google Colab, you need to restart your kernel after installation
# You can skip this step in Vertex AI Workbench

import sys

if "google.colab" in sys.modules:
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)


In [1]:
# If you are running this notebook on Google Colab, run the cell below to authenticate your environment
# If you are not using Google Colab, ensure that your Jupyter Notebook instance is connected to Google Cloud
# See https://cloud.google.com/sdk/gcloud for more information

import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [2]:
# Set your GCP project and region

import vertexai

PROJECT_ID = "genai-app-bldr"  # @param {type:"string"}
REGION = "asia-northeast3"  # @param {type:"string"}

# Initialize Vertex AI SDK
vertexai.init(project=PROJECT_ID, location=REGION)

In [5]:
# Import the class to interact with a Google Cloud Vertex AI Chatbot and set up a gemini-flash model

from langchain_google_vertexai import ChatVertexAI

model = ChatVertexAI(model="gemini-2.0-flash", location="us-central1")

Let's start with a simple conversation. Conversations are always an exchange of `HumanMessage` and `AIMessage`. The `HumanMessage` is provided by the user, while the `AIMessage` is generated by the model. When you print the full `AIMessage`, you will see that it comes with a lot of metadata, such as safety and usage information.

In [ ]:
# First, we send a HumanMessage
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hello, my name is Max!")])

In [ ]:
# Unfortunately, our chatbot does not have memory, and will not remember previous messages
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="What is my name?")])

In [ ]:
# To work around this, we can manually send the conversation history to the bot
from langchain_core.messages import HumanMessage, AIMessage

model.invoke([
    HumanMessage(content="Hello, my name is Max!"),
    AIMessage(content="Hello Max, it's a pleasure to meet you. How can I assist you today?"),
    HumanMessage("What is my name?")
    ])

Sending the full message conversation on every invocation is cumbersome and requires a lot of coding work on your part. Luckily, Langchain have released Langgraph, which handles memory using the checkpoint class.

In [ ]:
# Import necessary langgraph classes to define the chatbot and handle memory for you
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_vertexai import ChatVertexAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages

from typing import Annotated

from typing_extensions import TypedDict

# Create the agent

class State(TypedDict):
    # update add_messages to append messages not overwrite
    messages: Annotated[list, add_messages]

class StateMachine:

    def __init__(self, memory):

        model = ChatVertexAI(model="gemini-1.5-pro")

        # Define a system prompt that defines how the agent should interact with users
        system_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """You are an expert advisor in a bank. Your name is Terry.
                You advise users on the three loan products of the bank:
                1/ Short term loan, 6 month duration, 5% interest rate, up to USD 10000. Great for making quick payments.
                2/ Mid-term loan, 24 month duration, 3% interest rate, up to USD 20000. Great for buying used cars.
                3/ Long-term loan, 60 month duration, 2% interest rate, up to USD 50000. Great for buying a new car or a cheap house.
                Only respond to user questions on these loans.
                    """
            ),
            ("placeholder", "{messages}"),
            ("human", "{input}"),
        ]
        )

        # Bind the system_prompt to the model
        sally = system_prompt | model

        def chatbot(state: State):
            return {"messages": [sally.invoke(state["messages"])]}\

        graph_builder = StateGraph(State)

        graph_builder.add_node("chatbot", chatbot)

        # Set a finish point. This instructs the graph "any time this node is run, you can exit."
        graph_builder.set_finish_point("chatbot")
        graph_builder.set_entry_point("chatbot")

        # If you use memory, you need to provide a thread id to the chatbot
        self.thread = {"configurable": {"thread_id": "2"}}
        self.chain = graph_builder.compile(checkpointer=memory)

    def respond(self, USER_MESSAGE:str):
        result = self.chain.invoke({"messages": ("user", USER_MESSAGE)}, self.thread)

        # Need to cut off the first 80 characters, which say "AI Message"
        return result["messages"][-1].pretty_repr()[80:]



# Create memory for the bot
with SqliteSaver.from_conn_string(":memory:") as memory:
  # Initiate the memory
  chatbot = StateMachine(memory)

  # Draw the graph
  chatbot.chain.get_graph().print_ascii()

  print("Agent will keep going until you say 'STOP'")
  question = ""
  while "STOP" not in question:
      question = input("User: ")
      answer = chatbot.respond(question)
      print("Bot: " + answer)

